<a href="https://colab.research.google.com/github/rael395/Credidit-Scoring-App/blob/main/Code_CreditScoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Importing libraries & functions





In [19]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.linear_model import LogisticRegression

### Importing dataset

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
%cd /content/drive/MyDrive/Colab/Project/MiCred/Datasets
# !pwd

/content/drive/MyDrive/Colab/Project/MiCred/Datasets


In [22]:
df=pd.read_csv("cred_score.csv")
# !ls

### Data preparation

In [23]:
# shows count of rows and columns
df.shape

(3000, 30)

In [24]:
#shows first few rows of the code
df.head()

,TARGET,ID,DerogCnt,CollectCnt,BanruptcyInd,InqCnt06,InqTimeLast,InqFinanceCnt24,TLTimeFirst,TLTimeLast,...,TL50UtilCnt,TLBalHCPct,TLSatPct,TLDel3060Cnt24,TLDel90Cnt24,TLDel60CntAll,TLOpenPct,TLBadDerogCnt,TLDel60Cnt24,TLOpen24Pct
0,1,582,3,3,0,4,0.0,5,117,27,...,3.0,92%,21%,2,3,7,21%,4,4,0%
1,1,662,15,9,0,3,1.0,3,14,14,...,1.0,80%,0%,0,0,0,100%,12,0,100%
2,1,805,0,0,0,1,5.0,1,354,7,...,5.0,36%,65%,0,1,1,73%,1,1,53%
3,1,1175,8,5,0,6,1.0,10,16,4,...,3.0,91%,25%,1,1,1,75%,7,1,133%
4,1,1373,3,1,0,9,0.0,8,130,52,...,1.0,125%,0%,0,1,4,14%,3,1,0%


In [25]:
#dropping customer ID column from the dataset
df=df.drop('ID',axis=1)
df.shape

(3000, 29)

In [26]:
# explore missing values
df.isna().sum()

TARGET               0
DerogCnt             0
CollectCnt           0
BanruptcyInd         0
InqCnt06             0
InqTimeLast        188
InqFinanceCnt24      0
TLTimeFirst          0
TLTimeLast           0
TLCnt03              0
TLCnt12              0
TLCnt24              0
TLCnt                3
TLSum               40
TLMaxSum            40
TLSatCnt             4
TLDel60Cnt           0
TLBadCnt24           0
TL75UtilCnt         99
TL50UtilCnt         99
TLBalHCPct          41
TLSatPct             4
TLDel3060Cnt24       0
TLDel90Cnt24         0
TLDel60CntAll        0
TLOpenPct            3
TLBadDerogCnt        0
TLDel60Cnt24         0
TLOpen24Pct          3
dtype: int64

In [27]:
# filling missing values with mean
df=df.fillna(df.mean())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  


In [28]:
# explore missing values post missing value fix
df.isna().sum()

TARGET              0
DerogCnt            0
CollectCnt          0
BanruptcyInd        0
InqCnt06            0
InqTimeLast         0
InqFinanceCnt24     0
TLTimeFirst         0
TLTimeLast          0
TLCnt03             0
TLCnt12             0
TLCnt24             0
TLCnt               0
TLSum              40
TLMaxSum           40
TLSatCnt            0
TLDel60Cnt          0
TLBadCnt24          0
TL75UtilCnt         0
TL50UtilCnt         0
TLBalHCPct         41
TLSatPct            4
TLDel3060Cnt24      0
TLDel90Cnt24        0
TLDel60CntAll       0
TLOpenPct           3
TLBadDerogCnt       0
TLDel60Cnt24        0
TLOpen24Pct         3
dtype: int64

In [29]:
df= df.dropna(axis=1)

In [30]:
df.isna().sum()

TARGET             0
DerogCnt           0
CollectCnt         0
BanruptcyInd       0
InqCnt06           0
InqTimeLast        0
InqFinanceCnt24    0
TLTimeFirst        0
TLTimeLast         0
TLCnt03            0
TLCnt12            0
TLCnt24            0
TLCnt              0
TLSatCnt           0
TLDel60Cnt         0
TLBadCnt24         0
TL75UtilCnt        0
TL50UtilCnt        0
TLDel3060Cnt24     0
TLDel90Cnt24       0
TLDel60CntAll      0
TLBadDerogCnt      0
TLDel60Cnt24       0
dtype: int64

In [31]:
# count of good loans (0) and bad loans (1)
df['TARGET'].value_counts()

0    2500
1     500
Name: TARGET, dtype: int64

In [32]:
#data summary across 0 & 1
df.groupby('TARGET').mean()

,DerogCnt,CollectCnt,BanruptcyInd,InqCnt06,InqTimeLast,InqFinanceCnt24,TLTimeFirst,TLTimeLast,TLCnt03,TLCnt12,...,TLSatCnt,TLDel60Cnt,TLBadCnt24,TL75UtilCnt,TL50UtilCnt,TLDel3060Cnt24,TLDel90Cnt24,TLDel60CntAll,TLBadDerogCnt,TLDel60Cnt24
TARGET,,,,,,,,,,,,,,,,,,,,,
0,1.3224,0.7664,0.1492,2.9424,3.174638,3.2896,173.002,11.650,0.2844,1.832,...,13.822409,1.2884,0.470,3.015435,3.986711,0.6044,0.6624,2.2236,1.180,0.8648
1,1.9680,1.3100,0.1740,3.9380,2.775459,4.8820,155.672,12.992,0.2280,1.768,...,11.958047,2.6900,1.052,3.652920,4.533870,1.3340,1.5760,4.0140,2.554,2.0860


### Train Test Split

In [33]:
y = df.iloc[:, 0].values
X = df.iloc[:, 1:29].values

In [34]:
# splitting dataset into training and test (in ratio 80:20)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [35]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [36]:
#mount google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [37]:
# Exporting Normalisation Coefficients for later use in prediction
import joblib
joblib.dump(sc, '/content/drive/My Drive/Colab/Project/MiCred/Normal_CreditScoring')

['/content/drive/My Drive/Colab/Project/MiCred/Normal_CreditScoring']

### Risk Model building

In [38]:
classifier =  LogisticRegression()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

In [39]:
# Exporting Logistic Regression Classifier for later use in prediction

# import joblib
joblib.dump(classifier, '/content/drive/My Drive/Colab/Project/MiCred/Classifier_CreditScoring')

['/content/drive/My Drive/Colab/Project/MiCred/Classifier_CreditScoring']

### Model *performance*

In [40]:
print(confusion_matrix(y_test,y_pred))

[[485  13]
 [ 86  16]]


In [41]:
print(accuracy_score(y_test, y_pred))

0.835


### Writing output file

In [42]:
predictions = classifier.predict_proba(X_test)
predictions

array([[0.02664542, 0.97335458],
       [0.93964336, 0.06035664],
       [0.84714736, 0.15285264],
       ...,
       [0.88624041, 0.11375959],
       [0.46050172, 0.53949828],
       [0.91535579, 0.08464421]])

In [43]:
# writing model output file

df_prediction_prob = pd.DataFrame(predictions, columns = ['prob_0', 'prob_1'])
df_prediction_target = pd.DataFrame(classifier.predict(X_test), columns = ['predicted_TARGET'])
df_test_df = pd.DataFrame(y_test,columns= ['Actual Outcome'])

dfx=pd.concat([df_test_df, df_prediction_prob, df_prediction_target], axis=1)

dfx.to_csv("/content/drive/My Drive/Colab/Project/MiCred/Model_Prediction.xlsx", sep=',', encoding='UTF-8')

dfx.head()

,Actual Outcome,prob_0,prob_1,predicted_TARGET
0,1,0.026645,0.973355,1
1,0,0.939643,0.060357,0
2,0,0.847147,0.152853,0
3,0,0.885553,0.114447,0
4,0,0.818641,0.181359,0


### Coding ends here!